In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Încarcă setul de date
data = pd.read_csv('12000_lyrics_dataset.csv')

# Elimină rândurile cu valori NaN în coloana 'Cleaned_Lyrics'
data = data.dropna(subset=['Cleaned_Lyrics'])

# Initializează vectorizatorul și modelul LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
lda = LatentDirichletAllocation(n_components=10, random_state=0)

# Vectorizează textele din coloana 'Cleaned_Lyrics'
X = vectorizer.fit_transform(data['Cleaned_Lyrics'])

# Antrenează modelul LDA pe datele vectorizate
lda.fit(X)

# Obține distribuția topic-urilor pentru fiecare instanță de versuri
topic_distributions = lda.transform(X)

# Identifică topic-ul dominant pentru fiecare instanță
dominant_topics = topic_distributions.argmax(axis=1)

# Adaugă topic-ul dominant la setul de date original
data['topic_dominant'] = dominant_topics

# Afișează rezultatele inițiale
print(data[['Cleaned_Lyrics', 'Genre', 'topic_dominant']])

# Funcție pentru a obține cuvintele cele mai relevante pentru fiecare topic
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words[topic_idx] = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    return top_words

# Obține cuvintele corespunzătoare fiecărui topic
feature_names = vectorizer.get_feature_names_out()
top_words = get_top_words(lda, feature_names, 20)

# Afișează cuvintele pentru fiecare topic (fără filtrare)
print("Cuvintele pentru fiecare topic (fără filtrare):")
for topic, words in top_words.items():
    print(f"Topic {topic}: {', '.join(words)}")

# Salvează cuvintele pentru fiecare topic (fără filtrare) într-un fișier CSV
with open('12000_top_words_without_filtering.csv', 'w') as f:
    f.write("Topic,Words\n")
    for topic, words in top_words.items():
        f.write(f"{topic},{', '.join(words)}\n")

# Identifică cuvintele comune între topicuri
word_counts = {}
for words in top_words.values():
    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1

# Elimină cuvintele comune între topicuri
unique_top_words = {}
for topic_idx, words in top_words.items():
    unique_top_words[topic_idx] = [word for word in words if word_counts[word] == 1]

# Afișează cuvintele filtrate pentru fiecare topic
print("Cuvintele pentru fiecare topic (cu filtrare):")
for topic, words in unique_top_words.items():
    print(f"Topic {topic}: {', '.join(words)}")

# Salvează cuvintele pentru fiecare topic (cu filtrare) într-un fișier CSV
with open('12000_top_words_with_filtering.csv', 'w') as f:
    f.write("Topic,Words\n")
    for topic, words in unique_top_words.items():
        f.write(f"{topic},{', '.join(words)}\n")

# Analiza frecvenței topicurilor pe genuri
topic_genre_distribution = data.groupby(['Genre', 'topic_dominant']).size().unstack(fill_value=0)

# Afișează distribuția topicurilor pe genuri
print(topic_genre_distribution)

# Salvează distribuția topicurilor pe genuri într-un fișier CSV
topic_genre_distribution.to_csv('12000_topic_genre_distribution.csv')

# Salvează setul de date original cu topicurile dominante într-un fișier CSV
data.to_csv('12000_lyrics_with_topics.csv', index=False)


                                          Cleaned_Lyrics  Genre  \
0      Red dragon from the first morning of time\nRed...  Metal   
1      Broad incision sits across the evening\nA vict...  Metal   
2      I dont wanna see\nI dont wanna say\nTime stand...  Metal   
3      Weve walked together down this winding road\nI...  Metal   
4      Brothers the battle is raging choose your side...  Metal   
...                                                  ...    ...   
47995  Intro\nYou did it baby Congratulations homie\n...    Rap   
47996  Tah Murdah\nPerminently dedicated to the stree...    Rap   
47997  Intro\nCompton in the house\n\n\nChorus DJ Qui...    Rap   
47998  Bizzy\nThere is no way in hell marching factio...    Rap   
47999  Produced by Tae Beast\n\nIntro\nMake sure the ...    Rap   

       topic_dominant  
0                   3  
1                   9  
2                   1  
3                   1  
4                   9  
...               ...  
47995               8  
479

In [5]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pyLDAvis gensim

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim
from gensim.matutils import Sparse2Corpus
from gensim.corpora import Dictionary


# Convert CountVectorizer result to a Gensim corpus and dictionary
corpus = Sparse2Corpus(X, documents_columns=False)
id2word = Dictionary.from_corpus(corpus, id2word=dict((i, s) for i, s in enumerate(vectorizer.get_feature_names_out())))

# Vizualizarea topicurilor cu pyLDAvis
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(lda, corpus, id2word, sort_topics=False)

# Afișează vizualizarea
pyLDAvis.show(lda_display)

# Salvează vizualizarea în format HTML pentru a putea fi accesată ulterior
pyLDAvis.save_html(lda_display, '12000_lda_visualization.html')

AttributeError: 'LatentDirichletAllocation' object has no attribute 'num_topics'

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim
from gensim.matutils import Sparse2Corpus
from gensim.corpora import Dictionary

# Încarcă setul de date
data = pd.read_csv('12000_lyrics_dataset.csv')

# Elimină rândurile cu valori NaN în coloana 'Cleaned_Lyrics'
data = data.dropna(subset=['Cleaned_Lyrics'])

# Initializează vectorizatorul și modelul LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
lda = LatentDirichletAllocation(n_components=10, random_state=0)

# Vectorizează textele din coloana 'Cleaned_Lyrics'
X = vectorizer.fit_transform(data['Cleaned_Lyrics'])

# Antrenează modelul LDA pe datele vectorizate
lda.fit(X)

# Obține distribuția topic-urilor pentru fiecare instanță de versuri
topic_distributions = lda.transform(X)

# Identifică topic-ul dominant pentru fiecare instanță
dominant_topics = topic_distributions.argmax(axis=1)

# Adaugă topic-ul dominant la setul de date original
data['topic_dominant'] = dominant_topics

# Afișează rezultatele inițiale
print(data[['Cleaned_Lyrics', 'Genre', 'topic_dominant']])

# Funcție pentru a obține cuvintele cele mai relevante pentru fiecare topic
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words[topic_idx] = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    return top_words

# Obține cuvintele corespunzătoare fiecărui topic
feature_names = vectorizer.get_feature_names_out()
top_words = get_top_words(lda, feature_names, 20)

# Afișează cuvintele pentru fiecare topic (fără filtrare)
print("Cuvintele pentru fiecare topic (fără filtrare):")
for topic, words in top_words.items():
    print(f"Topic {topic}: {', '.join(words)}")

# Salvează cuvintele pentru fiecare topic (fără filtrare) într-un fișier CSV
with open('12000_top_words_without_filtering.csv', 'w') as f:
    f.write("Topic,Words\n")
    for topic, words in top_words.items():
        f.write(f"{topic},{', '.join(words)}\n")

# Identifică cuvintele comune între topicuri
word_counts = {}
for words in top_words.values():
    for word in words:
        word_counts[word] = word_counts.get(word, 0) + 1

# Elimină cuvintele comune între topicuri
unique_top_words = {}
for topic_idx, words in top_words.items():
    unique_top_words[topic_idx] = [word for word in words if word_counts[word] == 1]

# Afișează cuvintele filtrate pentru fiecare topic
print("Cuvintele pentru fiecare topic (cu filtrare):")
for topic, words in unique_top_words.items():
    print(f"Topic {topic}: {', '.join(words)}")

# Salvează cuvintele pentru fiecare topic (cu filtrare) într-un fișier CSV
with open('12000_top_words_with_filtering.csv', 'w') as f:
    f.write("Topic,Words\n")
    for topic, words in unique_top_words.items():
        f.write(f"{topic},{', '.join(words)}\n")

# Analiza frecvenței topicurilor pe genuri
topic_genre_distribution = data.groupby(['Genre', 'topic_dominant']).size().unstack(fill_value=0)

# Afișează distribuția topicurilor pe genuri
print(topic_genre_distribution)

# Salvează distribuția topicurilor pe genuri într-un fișier CSV
topic_genre_distribution.to_csv('12000_topic_genre_distribution.csv')

# Salvează setul de date original cu topicurile dominante într-un fișier CSV
data.to_csv('12000_lyrics_with_topics.csv', index=False)

# Convert CountVectorizer result to a Gensim corpus and dictionary
corpus = Sparse2Corpus(X, documents_columns=False)
id2word = Dictionary.from_corpus(corpus, id2word=dict((i, s) for i, s in enumerate(vectorizer.get_feature_names_out())))

# Vizualizarea topicurilor cu pyLDAvis
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(lda, corpus, id2word, sort_topics=False)

# Afișează vizualizarea
pyLDAvis.show(lda_display)

# Salvează vizualizarea în format HTML pentru a putea fi accesată ulterior
pyLDAvis.save_html(lda_display, '12000_lda_visualization.html')


                                          Cleaned_Lyrics  Genre  \
0      Red dragon from the first morning of time\nRed...  Metal   
1      Broad incision sits across the evening\nA vict...  Metal   
2      I dont wanna see\nI dont wanna say\nTime stand...  Metal   
3      Weve walked together down this winding road\nI...  Metal   
4      Brothers the battle is raging choose your side...  Metal   
...                                                  ...    ...   
47995  Intro\nYou did it baby Congratulations homie\n...    Rap   
47996  Tah Murdah\nPerminently dedicated to the stree...    Rap   
47997  Intro\nCompton in the house\n\n\nChorus DJ Qui...    Rap   
47998  Bizzy\nThere is no way in hell marching factio...    Rap   
47999  Produced by Tae Beast\n\nIntro\nMake sure the ...    Rap   

       topic_dominant  
0                   3  
1                   9  
2                   1  
3                   1  
4                   9  
...               ...  
47995               8  
479

AttributeError: 'LatentDirichletAllocation' object has no attribute 'num_topics'